# Cassandra

## Conexión con Cassandra y borrado de datos

In [1]:
%load_ext cql

In [2]:
%%cql
DROP KEYSPACE black;

'No results.'

In [3]:
%%cql
CREATE KEYSPACE black 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

'No results.'

In [4]:
%cql USE black;

'No results.'

## Creacción de las tablas

En **Cassandra** vamos a crear 2 tablas principales, una donde almacenamos los movimientos en formato desnormalizado, y otra que nos servirá para almacenar los importes acumulados por cliente.

También tenemos una tabla auxiliar, creada gracias a la funcionalidad que nos da la base de datos de VISTAS MATERIALIZADAS, que vamos a utilizar para facilitar otros patrones distintos de acceso a la información.

En la tabla que almacena los movimientos vamos a tener como **PARTITION_KEY** el campo nombre, ya que nos interesa tener en el mismo nodo todos los datos de una persona. Como **CUSTERING_KEY** vamos a tener el importe, debido a que en nuestro caso de uso queremos los datos ordenador por importe

<br><br> 

<img src="images/Modelo%20Cassandra.png" width="800" height="500">

<br><br> 

In [5]:
%%cql 
CREATE TABLE acum_movimientos_nombre (
    nombre          text,
    importe         counter,
    PRIMARY KEY (nombre)
)

'No results.'

In [6]:
%%cql 
CREATE TABLE movimientos (
    fecha           date,
    hora            int,
    minuto          int,
    importe         decimal,
    comercio        text,
    actividad_completa text,
    actividad       text,
    nombre          text,
    funcion         text,
    organizacion    text,
    PRIMARY KEY ((nombre), importe, fecha, hora, minuto)
)
WITH CLUSTERING ORDER BY (importe DESC);

'No results.'

En la vista materializada no es necesario insertar datos ya que se ocupa la propia base de datos

In [7]:
%%cql
CREATE MATERIALIZED VIEW vm_movimientos_by_actividad AS
   SELECT * FROM movimientos
   WHERE actividad IS NOT NULL 
         and importe IS NOT NULL 
         and fecha IS NOT NULL
         and hora IS NOT NULL 
         and minuto IS NOT NULL 
         and nombre IS NOT NULL
   PRIMARY KEY (actividad, importe, fecha,hora,minuto,nombre)
   WITH CLUSTERING ORDER BY (importe desc)

'No results.'

## Carga de datos en pandas

Cargamos los datos en formato Excel y los desnormalizamos ...

In [8]:
import pandas as pd
df_mov = pd.read_excel("../../data/black.xlsx", sheet_name= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheet_name= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_miembro          7624 non-null   int64         
 1   fecha               7624 non-null   datetime64[ns]
 2   hora                7624 non-null   int64         
 3   minuto              7624 non-null   int64         
 4   importe             7624 non-null   float64       
 5   comercio            6882 non-null   object        
 6   actividad_completa  7621 non-null   object        
 7   actividad           7621 non-null   object        
 8   nombre              7624 non-null   object        
 9   funcion             7624 non-null   object        
 10  organizacion        6060 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


### Carga de datos en Cassandra

In [9]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('black')

A destacar varios aspectos:
- El tipo de las fechas. Hay que dejarlo con el tipo **Date** de Python para no tener problemas (El tipo de la columna es también Date)
- Los nulos en Pandas son del tipo NumPy.nan, por lo hay convertirlos a None para que la inserción en Cassandra sea correcta
- En la tabla de importes acumulados guardamos el importe en un dato de tipo **counter**, que tiene un tipo entero, por lo que multiplicamos el dato por 100 para poder almacenarlo correctamete

In [10]:
df.loc[df.comercio.isnull(),['comercio']] = None
df.loc[df.organizacion.isnull(),['organizacion']] = None
df.loc[df.actividad.isnull(),['actividad']] = None
df.loc[df.actividad_completa.isnull(),['actividad_completa']] = None

In [11]:
import dateutil

def insert_movientos(df):
    
    sql_insert = """
INSERT INTO movimientos (
importe,
actividad,
fecha,
actividad_completa,
comercio,
funcion,
hora,
minuto,
organizacion,
nombre
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
"""

    for index in df.index:
        fecha = df['fecha'][index]
        
        data = [
            df["importe"][index],
            df["actividad"][index],
            fecha.date(),
            df["actividad_completa"][index],
            df["comercio"][index],
            df["funcion"][index],
            df["hora"][index],
            df["minuto"][index],
            df["organizacion"][index],
            df["nombre"][index],
        ]
        
        session.execute(sql_insert, data)
        
        importe_int = int(round(df["importe"][index] * 100))        
        session.execute("UPDATE acum_movimientos_nombre SET importe = importe + %s WHERE nombre = %s", 
                        [importe_int, 
                         df["nombre"][index]]
        )

Ya podemos insertar los datos ...

In [12]:
insert_movientos(df)

## Consulta de datos

Función de utilidad que realiza un query en Cassandra y devuelve un DataFrame de Pandas

In [13]:
def execute_query(sql):
    rows = session.execute(sql)
    return pd.DataFrame(list(rows))

### Los 10 movimientos mas caros

Para resolver esta query con la base de datos tendríamos que tener un PARTITION KEY única para todos los registros

In [14]:
sql = """
SELECT nombre, fecha, actividad_completa, importe
FROM MOVIMIENTOS
"""
df = execute_query(sql)

In [15]:
df.sort_values('importe', ascending = False).head(10)

,nombre,fecha,actividad_completa,importe
716,Enrique de la Torre Martínez,2007-11-28,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),12000.0
368,Ramón Ferraz Ricarte,2007-12-19,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),9424.08
717,Enrique de la Torre Martínez,2008-10-23,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),9000.0
718,Enrique de la Torre Martínez,2006-11-29,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),8000.0
4304,Miguel Blesa de la Parra,2006-12-26,EL CORTE INGLES,6397.31
1020,Rafael Spottorno Díaz Caro,2007-11-30,CONFECCION TEXTIL EN GENERAL,6375.0
2636,Carmen Contreras Gómez,2006-08-13,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",6198.02
5634,Matías Amat Roca,2006-08-13,BRITISH AIRWAYS - BRITISH A,5763.1
4881,Juan Manuel Astorqui Portera,2005-07-31,AUTOM.Y MOTOCICLETAS ( VENTAS Y REPARAC),5403.97
719,Enrique de la Torre Martínez,2004-12-06,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),5390.0


### Los movimientos de una persona concreta

Se resuelve la query con la base de datos, ya que la PARTITION KEY es el campo **nombre**, y los registros están ordenados por importe

In [16]:
sql = """
select nombre, fecha, actividad_completa, importe
from movimientos
where nombre = 'Javier de Miguel Sánchez'
limit 10
"""
execute_query(sql)

,nombre,fecha,actividad_completa,importe
0,Javier de Miguel Sánchez,2003-02-11,EL CORTE INGLES,1190.0
1,Javier de Miguel Sánchez,2007-10-12,AUTOM.Y MOTOCICLETAS ( VENTAS Y REPARAC),795.0
2,Javier de Miguel Sánchez,2009-07-18,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",571.61
3,Javier de Miguel Sánchez,2008-05-02,CONFECCION TEXTIL EN GENERAL,474.0
4,Javier de Miguel Sánchez,2008-11-28,CONFECCION TEXTIL EN GENERAL,389.0
5,Javier de Miguel Sánchez,2009-04-15,RESTAURANTES RESTO,355.2
6,Javier de Miguel Sánchez,2003-04-28,RESTAURANTES RESTO,338.01
7,Javier de Miguel Sánchez,2005-06-19,ZAPATERIAS,328.0
8,Javier de Miguel Sánchez,2006-12-08,EL CORTE INGLES,304.66
9,Javier de Miguel Sánchez,2009-09-11,CADENA MEDIA MARKT,299.0


### Las 10 personas que mas han gastado

Esta consulta se pueder resolver mediante el acumulado que habíamos creado ad-doc. Observa que el importe es entero, por lo que hay que dividirlo por 100 para convertirlo a decimal

In [17]:
sql = """
select *
from acum_movimientos_nombre
"""
df = execute_query(sql)

In [18]:
df.assign(importe = df.importe / 100).sort_values('importe', ascending = False).head(10)

,nombre,importe
11,Enrique de la Torre Martínez,58826.28
71,José Antonio Moral Santín,52447.07
4,Ramón Ferraz Ricarte,45371.52
54,Miguel Blesa de la Parra,40298.84
56,Ildefonso José Sánchez Barcoj,39437.99
58,Juan Manuel Astorqui Portera,36627.89
53,Francisco Baquero Noriega,35256.67
66,Matías Amat Roca,33434.74
34,Carmen Contreras Gómez,31556.45
20,Estanislao Rodríguez-Ponga Salamanca,29350.65


### Los 10 movimientos mas caros por actividad

Esta consulta se pueder resolver mediante la vista materializa que hemos creado ad-hoc para resolver esta consulta

In [19]:
sql = """
select nombre, fecha, actividad, actividad_completa, importe
from vm_movimientos_by_actividad
where actividad = 'HOGAR'
limit 10
"""
execute_query(sql)

,nombre,fecha,actividad,actividad_completa,importe
0,Miguel Ángel Araujo Serrano,2007-10-14,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",2150.0
1,Mariano Pérez Claver,2005-12-16,HOGAR,FLORES Y PLANTAS,1908.19
2,Domingo Navalmoral Sánchez,2004-01-09,HOGAR,"TAPICERIAS,ALFOMBRAS",1800.0
3,Ramón Ferraz Ricarte,2008-07-06,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1281.31
4,Miguel Ángel Araujo Serrano,2011-03-10,HOGAR,MUEBLES DE COCINA,1231.0
5,Francisco Baquero Noriega,2007-04-20,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1188.9
6,Francisco Baquero Noriega,2006-11-03,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1090.0
7,Pedro Bugidos Garay,2006-02-07,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1000.0
8,Ricardo Romero de Tejada y Picatoste,2009-04-06,HOGAR,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",1000.0
9,Jesús Pedroche Nieto,2011-05-24,HOGAR,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",1000.0
